# Section: Federated Learning

# Lesson: Introducing Federated Learning

<font color=red>Federated Learning is a technique for training Deep Learning models on data to which you do not have access.</font> Basically:

<font color=blue>Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.</font>

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [3]:
import syft as sy

W0804 15:22:34.053323 4482250176 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0804 15:22:34.458039 4482250176 deprecation_wrapper.py:119] From /Users/lek/miniconda3/envs/pysyft/lib/python3.6/site-packages/tf_encrypted/session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [4]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

<font color=red>The essence of Federated Learning is the ability to train models in parallel on a wide number of machines.</font> Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, <font color=red>instead of using Torch tensors - we're now going to work with **pointers** to tensors.</font> Let me show you what I mean. <font color=blue>First, let's create a "pretend" machine owned by a "pretend" person</font> - we'll call him Bob.

In [5]:
bob = sy.VirtualWorker(hook, id="bob")

In [9]:
bob._objects

{}

In [12]:
x = th.tensor([1,2,3,4,5])

In [13]:
x

tensor([1, 2, 3, 4, 5])

In [14]:
x = x.send(bob)

In [15]:
x

(Wrapper)>[PointerTensor | me:40613648927 -> bob:26688247646]

In [12]:
bob._objects

{34196116615: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #objects:1>

In [14]:
x.id_at_location

34196116615

In [15]:
x.id

9856897708

In [16]:
x.owner

<VirtualWorker id:me #objects:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:9856897708 -> bob:34196116615]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [6]:
alice = sy.VirtualWorker(hook, id='alice')

In [31]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [32]:
x_ptr = x.send(bob, alice)

In [34]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:33811829268 -> bob:71615211823]
	-> (Wrapper)>[PointerTensor | me:21197454072 -> alice:84746910559]

In [35]:
x_ptr.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:33811829268 -> bob:71615211823], 'alice': (Wrapper)>[PointerTensor | me:21197454072 -> alice:84746910559]}

In [36]:
x_ptr.child.child

{'bob': (Wrapper)>[PointerTensor | me:33811829268 -> bob:71615211823],
 'alice': (Wrapper)>[PointerTensor | me:21197454072 -> alice:84746910559]}

In [37]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [28]:
x = th.tensor([1,2,3,4,5]).send(bob,alice)

In [29]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [30]:
x

AttributeError: 'Tensor' object has no attribute 'child'

# Lesson: Introducing Remote Arithmetic

In [16]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [17]:
x

(Wrapper)>[PointerTensor | me:53228319963 -> bob:19737788221]

In [18]:
y

(Wrapper)>[PointerTensor | me:35544844799 -> bob:28771474875]

In [19]:
z = x + y

In [20]:
z

(Wrapper)>[PointerTensor | me:73802749525 -> bob:68055985599]

In [21]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [22]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:95305614851 -> bob:2140335042]

In [23]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [30]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [31]:
z = (x + y).sum()

In [32]:
z.backward()

(Wrapper)>[PointerTensor | me:74225222797 -> bob:77847815993]

In [33]:
x

(Wrapper)>[PointerTensor | me:81742380450 -> bob:87785438046]

In [34]:
x.grad

(Wrapper)>[PointerTensor | me:81742380450 -> bob:87785438046]::grad

In [35]:
x = x.get()

In [36]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [37]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [41]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [42]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob) 

In [44]:
pred = input.mm(weights)

In [45]:
loss = ((pred - target)**2).sum()

In [46]:
loss.backward()

(Wrapper)>[PointerTensor | me:89253468947 -> bob:29636463024]

In [47]:
weights.data.sub_(weights.grad * 0.1)
weights.grad *= 0

In [48]:
for i in range(10):
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)
    

tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)


# Lesson: Garbage Collection and Common Errors


In [49]:
bob = bob.clear_objects()

In [50]:
bob._objects

{}

In [51]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [52]:
bob._objects

{82386355543: tensor([1, 2, 3, 4, 5])}

In [53]:
del x

In [54]:
bob._objects

{}

In [55]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [56]:
x.child.garbage_collect_data

True

In [57]:
bob._objects

{50778400174: tensor([1, 2, 3, 4, 5])}

In [58]:
x = "asdf"

In [59]:
bob._objects

{}

In [60]:
Out

AttributeError: 'Tensor' object has no attribute 'child'

In [61]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [62]:
x

(Wrapper)>[PointerTensor | me:61134874995 -> bob:77586125283]

In [63]:
bob._objects

{77586125283: tensor([1, 2, 3, 4, 5])}

In [64]:
x = "asdf"

In [65]:
bob._objects

{77586125283: tensor([1, 2, 3, 4, 5])}

In [66]:
del x

In [67]:
bob._objects

{77586125283: tensor([1, 2, 3, 4, 5])}

In [76]:
bob = bob.clear_objects()
bob._objects

{}

In [77]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [78]:
bob._objects

{88893519886: tensor([1, 2, 3, 4, 5])}

In [71]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [72]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:26705049365 -> bob:66265877326]
Tensor B: tensor([1, 1, 1, 1, 1])

In [73]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [74]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [79]:
bob._objects

{88893519886: tensor([1, 2, 3, 4, 5])}

In [80]:
x = th.tensor([1,2,3,4,5]).send(bob)
bob._objects

{32490272800: tensor([1, 2, 3, 4, 5])}

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [81]:
from torch import nn, optim

In [82]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [83]:
# A Toy Model
model = nn.Linear(2,1)

In [84]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [85]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(4.6489)
tensor(0.5815)
tensor(0.1923)
tensor(0.1099)
tensor(0.0696)
tensor(0.0447)
tensor(0.0287)
tensor(0.0185)
tensor(0.0120)
tensor(0.0077)
tensor(0.0050)
tensor(0.0032)
tensor(0.0021)
tensor(0.0014)
tensor(0.0009)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)


In [86]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [87]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [88]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [89]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [90]:
train()

tensor(0.6145, requires_grad=True)
tensor(0.8275, requires_grad=True)
tensor(0.2243, requires_grad=True)
tensor(0.4006, requires_grad=True)
tensor(0.1634, requires_grad=True)
tensor(0.2310, requires_grad=True)
tensor(0.0960, requires_grad=True)
tensor(0.1351, requires_grad=True)
tensor(0.0556, requires_grad=True)
tensor(0.0792, requires_grad=True)
tensor(0.0322, requires_grad=True)
tensor(0.0465, requires_grad=True)
tensor(0.0186, requires_grad=True)
tensor(0.0274, requires_grad=True)
tensor(0.0108, requires_grad=True)
tensor(0.0162, requires_grad=True)
tensor(0.0063, requires_grad=True)
tensor(0.0096, requires_grad=True)
tensor(0.0036, requires_grad=True)
tensor(0.0057, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0003, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using <font color=blue>Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back.</font> However, at the end of the example we realized that <font color=blue>we needed to go a bit further to protect people privacy.</font> <font color=red>Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)</font>

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [91]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [92]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [93]:
x

(Wrapper)>[PointerTensor | me:5240933864 -> bob:44025685672]

In [94]:
x = x.send(alice)

In [95]:
bob._objects

{44025685672: tensor([1, 2, 3, 4, 5])}

In [96]:
alice._objects

{5240933864: (Wrapper)>[PointerTensor | alice:5240933864 -> bob:44025685672]}

In [97]:
x

(Wrapper)>[PointerTensor | me:62587857749 -> alice:5240933864]

In [98]:
y = x + x

In [99]:
y

(Wrapper)>[PointerTensor | me:77894596765 -> alice:85855785215]

In [100]:
bob._objects

{44025685672: tensor([1, 2, 3, 4, 5]),
 16856773246: tensor([ 2,  4,  6,  8, 10])}

In [101]:
alice._objects

{5240933864: (Wrapper)>[PointerTensor | alice:5240933864 -> bob:44025685672],
 85855785215: (Wrapper)>[PointerTensor | alice:85855785215 -> bob:16856773246]}

In [102]:
jon = sy.VirtualWorker(hook, id="jon")

In [103]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [104]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:alice #objects:3> while the other is on <VirtualWorker id:jon #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [105]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [106]:
x

(Wrapper)>[PointerTensor | me:65645194880 -> alice:73533220682]

In [107]:
bob._objects

{8765948397: tensor([1, 2, 3, 4, 5])}

In [108]:
alice._objects

{73533220682: (Wrapper)>[PointerTensor | alice:73533220682 -> bob:8765948397]}

In [109]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:73533220682 -> bob:8765948397]

In [110]:
bob._objects

{8765948397: tensor([1, 2, 3, 4, 5])}

In [111]:
alice._objects

{}

In [112]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [113]:
bob._objects

{}

In [114]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [115]:
bob._objects

{21464044695: tensor([1, 2, 3, 4, 5])}

In [116]:
alice._objects

{80370565291: (Wrapper)>[PointerTensor | alice:80370565291 -> bob:21464044695]}

In [117]:
del x

In [118]:
bob._objects

{}

In [119]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [120]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [121]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [122]:
bob._objects

{35643738139: tensor([1, 2, 3, 4, 5])}

In [123]:
alice._objects

{58612468547: (Wrapper)>[PointerTensor | alice:58612468547 -> bob:35643738139]}

In [124]:
x.remote_get()

(Wrapper)>[PointerTensor | me:85185361569 -> alice:58612468547]

In [125]:
bob._objects

{}

In [126]:
alice._objects

{58612468547: tensor([1, 2, 3, 4, 5])}

In [127]:
x

(Wrapper)>[PointerTensor | me:85185361569 -> alice:58612468547]

In [128]:
x.move(bob)

(Wrapper)>[PointerTensor | me:85185361569 -> bob:85185361569]

In [129]:
x

(Wrapper)>[PointerTensor | me:85185361569 -> bob:85185361569]

In [130]:
bob._objects

{85185361569: tensor([1, 2, 3, 4, 5])}

In [131]:
alice._objects

{}

In [137]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [138]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [139]:
x

(Wrapper)>[PointerTensor | me:89633766833 -> bob:49404856725]

In [140]:
bob._objects

{49404856725: tensor([1, 2, 3, 4, 5])}

In [141]:
alice._objects

{}

In [142]:
x.move(alice)

(Wrapper)>[PointerTensor | me:89633766833 -> alice:89633766833]

In [143]:
bob._objects

{}

In [144]:
alice._objects

{89633766833: tensor([1, 2, 3, 4, 5])}

In [145]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [146]:
bob._objects

{3336244019: tensor([1, 2, 3, 4, 5])}

In [147]:
alice._objects

{89633766833: tensor([1, 2, 3, 4, 5]),
 84015757201: (Wrapper)>[PointerTensor | alice:84015757201 -> bob:3336244019]}

In [148]:
x.remote_get()

(Wrapper)>[PointerTensor | me:31638981874 -> alice:84015757201]

In [149]:
bob._objects

{}

In [150]:
alice._objects

{89633766833: tensor([1, 2, 3, 4, 5]), 84015757201: tensor([1, 2, 3, 4, 5])}

In [151]:
x.move(bob)

(Wrapper)>[PointerTensor | me:31638981874 -> bob:31638981874]

In [152]:
x

(Wrapper)>[PointerTensor | me:31638981874 -> bob:31638981874]

In [153]:
bob._objects

{31638981874: tensor([1, 2, 3, 4, 5])}

In [154]:
alice._objects

{89633766833: tensor([1, 2, 3, 4, 5])}